## ETL Run Notebook  - Run scripts & Test w/ sql queries

### Run create_table.py to see if tables can be created property

- Make sure you open a RedShift cluster and make it publicly accessible

In [1]:
import psycopg2 

try:
    %run create_table.py 
except Exception as e:
    print(e)

CREATE TABLE IF NOT EXISTS naturaldisaster_stage(
        year int, 
        seq int, 
        glide varchar,
        disaster_group varchar,
        disaster_subgroup varchar,
        diaster_type varchar, 
        disaster_subtype varchar,
        disaster_subsubtype varchar,
        event_name varchar,
        country varchar,
        iso varchar,
        region varchar,
        continent varchar,
        location varchar(5000),
        origin varchar, 
        associated_dis varchar,
        associated_dis2 varchar,
        OFDA_response varchar, 
        appeal varchar,
        declaration varchar,
        aid_contribution FLOAT,
        dis_mag_value FLOAT,
        dis_mag_scale varchar,
        latitude varchar,
        longitude varchar,
        local_time varchar,
        river_basin varchar(1000),
        start_year BIGINT,
        start_month varchar,
        start_day int,
        end_year BIGINT,
        end_month varchar,
        end_day int,
        total_deaths FLOAT,
 

In [2]:
try:
    %run etl.py 
    
except Exception as e:
    print(e)
    


#### Test tables with some Queries

In [3]:
import pandas as pd
import boto3
import json
import psycopg2

In [4]:
config_conn = configparser.ConfigParser()
config_conn.read('dwh.cfg')

# Run a query

query = ('''
SELECT * 
FROM surfacetemp
''')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config_conn['CLUSTER'].values()))
df = pd.read_sql_query(query, conn)
conn.close()

df

,year,month,country,avg_temperature,avg_temperature_uncertainty
0,1745,2,Åland,-6.575,1.360
1,1748,6,Åland,NaN,NaN
2,1756,10,Åland,6.680,4.140
3,1766,10,Åland,6.512,1.803
4,1772,10,Åland,9.246,2.189
...,...,...,...,...,...
577457,1984,8,Zimbabwe,18.497,0.337
577458,1990,8,Zimbabwe,18.690,0.193
577459,1999,12,Zimbabwe,24.112,0.245
577460,2005,4,Zimbabwe,22.166,0.237
